This file is **very similar** to Preliminary-analysis-1.

In [56]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [57]:
data = pd.read_csv('consolidated-1.csv')

In [58]:
time_window = '3min' # This is the time window over which we will average call arrivals.
time_window_size = int(time_window[0])

In [59]:
# This is the time in seconds taken to complete a call(included post-processing).
data['Avg-time'] = data['Avg-time'].fillna(0) 

$\lambda$ is the number of calls arriving in an interval equal to the <code>time_window_size</code>.

In [60]:
data['lambda'] = data['ncalls'].rolling(window=time_window_size).mean() # Number of calls per 3 minutes.
# The rolling window size has nothing to do with the time window.

In [61]:
data.set_index('Create-time', inplace=True)

$\mu$ is the average time to answer a call. The unit is '<code>time_window</code>'.

In [62]:
data['mu'] = data['Avg-time']/(60 * time_window_size) # The term lambda/mu must be unit-less.

The traffic intensity $E$ is in Erlangs.

In [63]:
data['E'] = data['lambda']*data['mu']

In [64]:
def prob_of_wait(E, m):
    """
    E:      traffic
    m:      # agents.
    Reference: https://en.wikipedia.org/wiki/Erlang_(unit)
    """ 
    p = 1
               
    if m > E:   
        try:
            numerator = E**m/math.factorial(m) * m/(m - E)
            denominator = 0

            for i in range(m):
                denominator += E**i/math.factorial(i)

            denominator += numerator

            p = numerator/denominator
        except OverflowError:
            print(f'Overflow due to E = {E}, m = {m}')
        
    return p

def calculate_ASA(wait_prob, mu, nagents, E, avg_time):   
    if nagents > E:
        return wait_prob * mu/(nagents - E)
    else:
        return avg_time/(60 * time_window_size)
    
def find_nagents(E, m):
    start = m

    if math.isnan(E):
        return start
    
    if E > m:
        threshold = 0.8
        #start = int(E)
        while (prob_of_wait(E, start) > threshold) and (start <= E):
            start += 1            
        
    return start

def find_nagents_1(E, m, threshold_asa, mu, avg_time):
    start = m
             
    if not math.isnan(E):        
        wait_prob = prob_of_wait(E, m)        
        while calculate_ASA(wait_prob, mu, start, E, avg_time) > threshold_asa:
            start += 1
            
    return start


In [65]:
data['wait-prob'] = data.apply(lambda r: prob_of_wait(r['E'], r['nagents']), axis=1)

Refer to [point 15](https://www.callcentrehelper.com/erlang-c-formula-example-121281.htm) for the formula to compute ASA

In [66]:
data['asa'] = data.apply(lambda r: calculate_ASA(r['wait-prob'], r['mu'], r['nagents'], r['E'], r['Avg-time']), axis=1)

In [67]:
data.head()

,ncalls,Avg-time,Period,nagents,office-hour,Avg-calls,iso_day_of_week,lambda,mu,E,wait-prob,asa
Create-time,,,,,,,,,,,,
2020-09-21 09:06:00,1,79.00,2020-09-21,22,True,NaN,1,NaN,0.438889,NaN,1.00000,0.438889
2020-09-21 09:09:00,2,1161.00,2020-09-21,22,True,NaN,1,NaN,6.450000,NaN,1.00000,6.450000
2020-09-21 09:12:00,20,452.65,2020-09-21,22,True,7.666667,1,7.666667,2.514722,19.279537,0.44781,0.413943
2020-09-21 09:15:00,20,612.70,2020-09-21,22,True,14.000000,1,14.000000,3.403889,47.654444,1.00000,3.403889
2020-09-21 09:18:00,12,536.50,2020-09-21,22,True,17.333333,1,17.333333,2.980556,51.662963,1.00000,2.980556


In [68]:
data.reset_index(level=0, inplace=True)

In [69]:
X = data.apply(lambda r: find_nagents_1(r['E'], r['nagents'], 4, r['mu'], r['Avg-time']), axis = 1)
Y = X.rolling(window=5).median()
Y.reset_index(drop=True, inplace=True)
data['additional_nagents_4m'] = Y - data['nagents']
data['additional_nagents_4m'] = data['additional_nagents_4m'].clip(lower=0, axis=0)

In [71]:
X = data.apply(lambda r: find_nagents_1(r['E'], r['nagents'], 2, r['mu'], r['Avg-time']), axis = 1)
Y = X.rolling(window=5).median()
Y.reset_index(drop=True, inplace=True)
data['additional_nagents_2m'] = Y - data['nagents']
data['additional_nagents_2m'] = data['additional_nagents_2m'].clip(lower=0, axis=0)

In [72]:
data['additional_nagents_4m_pct'] = data['additional_nagents_4m']/data['nagents'] * 100
data['additional_nagents_2m_pct'] = data['additional_nagents_2m']/data['nagents'] * 100

In [73]:
data.to_csv('wait_prob_1.csv', index=False, float_format = '%.3f')

The next step is to build a time-series model for $E$.